# Introduction
This notebook marks warehouse data management exercise. The primary goal here is Exploratory Data Analysis (EDA) and clean the data. We will load the provided `warehouse_data_messy.csv` dataset and perform an initial investigation to understand its structure, identify potential data quality issues, and get a general feel for the data distributions.

Next we will address the data quality issues identified, such as missing values, incorrect data types, inconsistent formatting, potential duplicates, and outliers.



# 1. Setup and Data Loading

First, we import the necessary library (pandas) and load the dataset from the CSV file into a DataFrame.

In [ ]:
import pandas as pd

# Load the dataset from the specified CSV file path
file_path = "../data/warehouse_data_messy.csv"
df = pd.read_csv(file_path)

# 2. Initial Data Inspection

Let's start with some basic pandas functions to get an overview of the DataFrame

## 2.1. Dataframe Structure and Types
The `info()` method provides a concise summary of the DataFrame, including the number of non-null entries for each column and their data types (Dtypes). This helps identify columns with missing data and incorrect data types early on.

In [ ]:
# Display DataFrame info: column names, non-null counts, and data types
df.info()

## 2.2. Fetch rows

The `head()` method displays the first few rows of the DataFrame (default is 5). This gives a quick look at the actual data values and helps spot obvious issues like inconsistent formatting or placeholder values.

To fetch last rows use method `tail()`.

In [ ]:
# Show the first 5 rows of the DataFrame
df.head()

## 2.3. Descriptive Statistics

The `describe()` method generates descriptive statistics for numerical columns (count, mean, std, min, max, quartiles). Using include='all' extends this to object/categorical columns, showing count, unique values, top value, and frequency.

**Note:** This might raise errors or produce unexpected results if numerical columns contain non-numeric data, which is common in messy datasets.

In [ ]:
# Generate descriptive statistics for all columns
df.describe(include="all")

# 3. Automated Data Profiling

For a more in-depth and automated overview we use the `ydata-profiling` library. It generates an interactive report summarizing various aspects of the data, including distributions, correlations, missing values, and potential quality issues.

In [ ]:
# Import the ProfileReport class
from ydata_profiling import ProfileReport

# Create a profile report instance
profile = ProfileReport(df, title="Raw Warehouse Data Profile", explorative=True)
profile.to_notebook_iframe()

Start with exploring Overview section of the report. It let you quickly understand the data. 
In this step you should notice some issues with data.
- 11 duplicated records,
- 99 missing values.

Then familirize with each Variable in the dataset. It let you understand the content of the selected column.
For example, you can see that `Product ID` has no missing values, but `location` has 18. `Quantity` and `Price` has incorrect column type. 

In [ ]:
# You can save the report to an HTML file
profile_output_path = "warehouse_data_profile.html"
profile.to_file(profile_output_path)

# 4. Data cleaning 

## 4.1. Handling Missing Values

Missing data can occur in various forms (standard NaN, empty strings, custom placeholders). We need to identify and address them appropriately.

### 4.1.1. Standard Missing Values

You can read missing values from above report or calculate it with Pandas `isnull().sum()`

In [ ]:
# Count standard NaN values per column
standard_missing = df.isnull().sum()
print("Standard Missing Values (NaN):\n", standard_missing[standard_missing > 0])

### 4.1.2. Non-standard Missing Values

Often, missing data is represented by strings like "unknown", "N/A", "--", etc. We need to define these patterns and search for them, typically in object-type columns.

In [ ]:
# Define patterns representing missing data
missing_patterns = [
    "--",
    "N/N",
    "unknown",
    "?",
    "None",
    "NA",
    "",
    "not available",
    "null",
]  # Added 'not available' and 'null'

# Create a boolean mask for non-standard missing values across the dataframe
non_standard_missing_mask = df.map(lambda x: str(x).strip() in missing_patterns)

# Count non-standard missing values per column
non_standard_missing_counts = non_standard_missing_mask.sum()

print(
    "Non-Standard Missing Values (Counts):\n",
    non_standard_missing_counts[non_standard_missing_counts > 0],
)

Now, replace the identified non-standard missing values with the standard np.nan. This makes subsequent handling easier.

In [ ]:
import numpy as np

# Replace non-standard patterns with np.nan
# We iterate through columns and apply replacement only where the mask is True
for col in df.columns:
    # Check if the column actually had any non-standard missing values to replace
    if non_standard_missing_counts.get(col, 0) > 0:
        df.loc[non_standard_missing_mask[col], col] = np.nan

# Verify total missing values now (should include both original NaN and replaced patterns)
total_missing = df.isnull().sum()
print(
    "Total Missing Values (NaN after consolidation):\n",
    total_missing[total_missing > 0],
)

### Imputation 

We need to decide how to fill the NaN values. Common strategies include:

- **Mean/median/mode or most common value imputation**: Simple, but can distort variance and correlations,
- **Forward/Backward fill**: Useful for time-series or ordered data,
- **Constant value**: Filling with a specific value (like 0, "Unknown"),
- **Dropping**: Removing rows or columns with missing data (use cautiously),
- **Model-based imputation (e.g., IterativeImputer)**: More complex, uses other features to predict missing values.

Strategy Chosen Here:

- `Quantity`: Impute with 0 (assuming missing means zero stock, but review this assumption).
- `Price`: Impute with the median, as it's less sensitive to outliers than the mean (we need to convert to numeric first).
- `Location`: Impute with a placeholder like "Unknown".
- `Status`: Impute with the mode (most frequent value).
- `Last Restocked`: Impute with a specific placeholder date or use ffill/bfill if ordering makes sense (we'll use a placeholder here after date conversion).

### Imputing categorical columns (Location, Status)

In [ ]:
# Impute Location with 'Unknown'
df["Location"] = df["Location"].fillna("Unknown")
print("Imputed 'Location' missing values with 'Unknown'.")

# Impute Status with the mode
status_mode = df["Status"].mode()[0]  # mode() returns a Series, get the first element
df["Status"] = df["Status"].fillna(status_mode)
print(f"Imputed 'Status' missing values with mode: '{status_mode}'.")

### Imputing numerical columns (Quantity, Price)

First, convert these columns to numeric types, coercing errors. Values that cannot be converted (like remaining non-standard patterns missed earlier, or text) will become `NaN`.

In [ ]:
# Convert Quantity and Price to numeric, errors become NaN
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

# Check for any new NaNs introduced by coercion
print(
    "Missing values after numeric conversion:\n",
    df[["Quantity", "Price"]].isnull().sum(),
)

#### Impute Price with the median

In [ ]:
price_median = df["Price"].median()
df["Price"] = df["Price"].fillna(price_median)
print(f"Imputed 'Price' missing values with median: {price_median:.2f}")

#### Impute Quantity with MICE algorithm

MICE (Multiple Imputation by Chained Equations) is an AI method for handling missing data. That works by:
1. Initially filling missing values with simple mean/median imputation. This is a temporary replacement. At the end of this step, there should be no missing values.
2. For the specific column you want to impute, eg: columm A alone, change the imputed value back to missing.
3. Now, build a regression model to predict A using (B and C) as predictors. For this model, only the non-missing rows of A are included. So, A is the response, while, B and C are predictors. Use this model to predict the missing values in A.
4. Repeat steps 2-3 for columns B and C as well.

MICE works with only numerical features. What can we do in this situation?
We can transform categorical features (such as Category, Warehouse, Supplier) to numeric using label encoding. Each unique value will be transformed to unique numeric identifier.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
category_numeric = le.fit_transform(df["Category"])
print(
    "Categories and their numeric encodings:\n",
    dict(zip(le.classes_, le.transform(le.classes_))),
)

In [ ]:
# Convert rest of the categorical columns to numeric
# Skip Product ID as it is unique for each product and not useful for predicting missing values

features = [
    "Product Name",
    "Category",
    "Warehouse",
    "Location",
    "Price",
    "Supplier",
    "Status",
    "Quantity",
]

# Create a copy of the dataframe with selected features
df_for_imputation = df[features].copy()

# Convert categorical variables to numeric using label encoding
# we saved the label encoder instance to reuse it for backward transformation
encoders = {}
for col in df_for_imputation:
    if df_for_imputation[col].dtype == "object":
        le = LabelEncoder()
        encoders[col] = le
        df_for_imputation[col] = le.fit_transform(df_for_imputation[col])

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Define imputer
imputer = IterativeImputer()

# fit on the dataset
imputer.fit(df_for_imputation)

imputed_array = imputer.transform(df_for_imputation)
df_imputed = pd.DataFrame(imputed_array, columns=df_for_imputation.columns)

# use the saved encoders to convert back to original categorical values
for col, le in encoders.items():
    df_imputed[col] = le.inverse_transform(df_imputed[col].astype(int))

# check number of missing values in Quantity column
print(
    "Missing values in 'Quantity' after imputation:\n",
    df_imputed["Quantity"].isnull().sum(),
)

# copy quanity column to original dataframe
df["Quantity"] = df_imputed["Quantity"]

### Handling Last Restocked (Date Column)

Convert to datetime objects, handling potential errors and different formats. Then impute.

In [ ]:
# Convert 'Last Restocked' to datetime, coercing errors
# The 'infer_datetime_format=True' can help with mixed formats, but might be slow.
# For truly mixed/complex formats, might need custom parsing logic.
df["Last Restocked"] = pd.to_datetime(df["Last Restocked"], errors="coerce")

# Check for NaNs introduced by conversion or already present
print(
    f"Missing 'Last Restocked' values before imputation: {df['Last Restocked'].isnull().sum()}\n"
)

# Imputation Strategy: Fill with a placeholder date (e.g., earliest date or a sentinel value)
# Or use ffill/bfill if data is chronologically ordered (not assumed here).
# Let's use the overall minimum date found in the column as placeholder.
min_date = df["Last Restocked"].min()
df["Last Restocked"] = df["Last Restocked"].fillna(min_date)
print(
    f"Imputed 'Last Restocked' missing values with min date: {min_date}. Note: Review if this is appropriate."
)

### Final Check for Missing Values

In [ ]:
# Verify that there are no more missing values
missing_after_imputation = df.isnull().sum().sum()
if missing_after_imputation == 0:
    print("Success: No missing values remaining in the dataset.")
else:
    print(f"Warning: {missing_after_imputation} missing values still present.")
    print(df.isnull().sum())

## 4.2 Correcting data types 

Ensure all columns have the appropriate data types for analysis.

- `Product ID`: Should ideally be string/object if not used mathematically, to avoid accidental calculations. Let's keep as int64 for now, assuming it's a simple ID.
- `Quantity`: Should be integer type now that it's cleaned and imputed.
- `Price`: Should be float type.
- `Last Restocked`: Should be datetime type.

In [ ]:
# Convert Quantity to integer (assuming whole units)
# Use Int64 (capital I) to allow for potential NaNs if any were missed, though we aimed to fill all.
try:
    df["Quantity"] = df["Quantity"].astype("Int64")
except Exception as e:
    print(f"Could not convert Quantity to Int64: {e}. Check for non-integer values.")
    # Fallback or further investigation needed if conversion fails

# Price is already float (implicitly from to_numeric and median)
# Last Restocked is already datetime

# Display final data types
print("\nFinal Data Types:\n", df.dtypes)

## 4.3. Handling inconsistent formatting

Clean up string columns by removing leading/trailing whitespace and extra quotes.

In [ ]:
# Select object type columns (strings)
string_columns = df.select_dtypes(include=["object"]).columns

for col in string_columns:
    # Remove leading/trailing whitespace
    df[col] = df[col].str.strip()
    # Remove potential extra quotes (be careful not to remove legitimate internal quotes if any)
    df[col] = df[col].str.replace(
        '^"|"$', "", regex=True
    )  # Removes quotes only at start/end

print("String columns cleaned (whitespace and edge quotes removed).")
# Display some examples from a potentially affected column like Product Name
print("\nExample cleaned 'Product Name':")
print(df["Product Name"].head())

## 4.4 Handling duplicates

Identify and remove duplicate rows.

In [ ]:
# Detect duplicate rows
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows found: {num_duplicates}")

In [ ]:
df.drop_duplicates(inplace=True)
print("Duplicate rows removed.")
# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [ ]:
# Check again if there are duplicated
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows found: {num_duplicates}")

## 4.3. Handling outliers

Outliers are extreme values that can skew analysis. We'll focus on Quantity and Price.

**Strategy:** Identify outliers using the Interquartile Range (IQR) method and decide whether to cap them (replace with a boundary value) or remove them. We'll visualize first, then cap.

### Visualize Potential Outliers

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns

# Create box plots to visualize distributions and potential outliers
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(y=df["Quantity"])
plt.title("Box Plot of Quantity")
plt.ylabel("Quantity")

plt.subplot(1, 2, 2)
sns.boxplot(y=df["Price"])
plt.title("Box Plot of Price")
plt.ylabel("Price")

plt.tight_layout()
plt.show()

The outliers are shown in the boxplots as dots above or below the upper/lower whisker. 

### Capping Outliers using IQR
We'll cap values below Q1 - 1.5IQR or above Q3 + 1.5IQR.

In [ ]:
def cap_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap values
    capped_series = series.clip(lower=lower_bound, upper=upper_bound)
    num_capped = (series < lower_bound).sum() + (series > upper_bound).sum()
    print(
        f"Capped {num_capped} outliers in '{series.name}' (Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f})."
    )
    return capped_series


# Apply capping to Quantity and Price
df["Quantity"] = cap_outliers(df["Quantity"])
df["Price"] = cap_outliers(df["Price"])

### Visualize After Capping

In [ ]:
# Visualize again after capping
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(y=df["Quantity"])
plt.title("Box Plot of Quantity (After Capping)")
plt.ylabel("Quantity")

plt.subplot(1, 2, 2)
sns.boxplot(y=df["Price"])
plt.title("Box Plot of Price (After Capping)")
plt.ylabel("Price")

plt.tight_layout()
plt.show()

# 5. Final Review and Saving Cleaned Data

Let's review the first few rows and the info of the cleaned DataFrame before saving.

In [ ]:
print("Cleaned DataFrame Info:")
df.info()

print("\nFirst 5 rows of Cleaned DataFrame:")
df.head()

In [ ]:
# Regenerate report on cleaned dataset to be sure that all issues are handled

profile = ProfileReport(df, title="Raw Warehouse Data Profile", explorative=True)
profile.to_notebook_iframe()

In [ ]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

In [ ]:
# 6. Synthetic Data Generation

from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(num_rows=50)
synthetic_data.head()

### Evaluating Real vs. Synthetic Data
SDV has built-in functions for evaluating the synthetic data and getting more insight.

As a first step, we can run a diagnostic to ensure that the data is valid. SDV's diagnostic performs some basic checks such as:

- All primary keys must be unique
- Continuous values must adhere to the min/max of the real data
- Discrete columns must have the same categories as the real data

In [ ]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=df, synthetic_data=synthetic_data, metadata=metadata
)

The score is 100%, indicating that the data is fully valid.

We can also measure the data quality or the statistical similarity between the real and synthetic data. This value may vary anywhere from 0 to 100%.

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(df, synthetic_data, metadata)

According to the score, the synthetic data is about 71% similar to the real data in terms of statistical similarity.

In [ ]:
df["Price"].plot.hist()

In [ ]:
synthetic_data["Price"].plot.hist()

In [ ]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv("../data/warehouse_data_cleaned.csv", index=False)

# 6. Conclusion
In this notebook, we performed essential data cleaning tasks on the warehouse inventory data. We addressed:

- Standard and non-standard missing values through identification and imputation,
- Incorrect data types by converting columns like Quantity, Price, and Last Restocked,
- Inconsistent string formatting by trimming whitespace and removing extraneous quotes,
- Duplicated records by identifying and removing them,
- Outliers by visualizing and capping them using the IQR method,
- Missing data imputation,
- Synthetic Data Generation.
The resulting DataFrame (df) is now cleaned and stored in `warehouse_data_cleaned.csv`. 